In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pickle

In [2]:
commands = [
    "turn on the led", "switch on light", "light on", "led on",
    "turn off the led", "switch off light", "light off", "led off"
]
labels = [1, 1, 1, 1, 0, 0, 0, 0]  # 1: ON, 0: OFF

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(commands)
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(commands)
padded = pad_sequences(sequences, padding='post')
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

In [4]:
labels = tf.keras.utils.to_categorical(labels, num_classes=2)

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=8, input_length=padded.shape[1]),
    tf.keras.layers.GRU(16),
    tf.keras.layers.Dense(2, activation='softmax')
])

C:\Users\Akash Potti\miniconda3\envs\tf\lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [8]:
model.summary()

# 5. Train
model.fit(padded, labels, epochs=100, verbose=0)

# 6. Save Weights
model.save_weights("gru_led_intent_weights.weights.h5")

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 4, 8)                │              64 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (None, 16)                  │           1,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 2)                   │              34 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,040 (15.79 KB)

 Trainable params: 1,346 (5.26 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,694 (10.53 KB)